## Домашнее задание

In [43]:
import random
from sklearn.datasets import make_regression
from sklearn.metrics import r2_score
import numpy as np

### Task 1. Реализуйте дерево для задачи регрессии. Возьмите за основу дерево, реализованное в методичке, заменив механизм предсказания в листе на взятие среднего значения по выборке, и критерий Джини на дисперсию значений.

In [15]:
# Реализуем класс узла

class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

In [16]:
# И класс терминального узла (листа)

class Leaf:
    
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets
        self.prediction = self.predict()
        
    def predict(self):   
        prediction = self.targets.mean()
        return prediction        

In [17]:
# Расчёт дисперсии значений
def mse(array):
    mean = array.mean()
    return np.mean((array - mean)**2)

In [19]:
# Расчет прироста

def gain(left_targets, right_targets, root_mse):

    # доля выборки, ушедшая в левое поддерево
    p = float(left_targets.shape[0]) / (left_targets.shape[0] + right_targets.shape[0])
    
    return root_mse - p * mse(left_targets) - (1 - p) * mse(right_targets)

In [20]:
# Разбиение датасета в узле

def split(data, targets, column_index, t):
    
    left = np.where(data[:, column_index] <= t)
    right = np.where(data[:, column_index] > t)
        
    true_data = data[left]
    false_data = data[right]
    
    true_targets = targets[left]
    false_targets = targets[right]
        
    return true_data, false_data, true_targets, false_targets

In [21]:
# Нахождение наилучшего разбиения

def find_best_split(data, targets):
    
    #  обозначим минимальное количество объектов в узле
    min_samples_leaf = 3

    root_mse = mse(targets)

    best_gain = 0
    best_t = None
    best_index = None
    
    n_features = data.shape[1]
    
    for index in range(n_features):
        # будем проверять только уникальные значения признака, исключая повторения
        t_values = np.unique(data[:, index])
        
        for t in t_values:
            true_data, false_data, true_targets, false_targets = split(data, targets, index, t)
            #  пропускаем разбиения, в которых в узле остается менее 5 объектов
            if len(true_data) < min_samples_leaf or len(false_data) < min_samples_leaf:
                continue
            
            current_gain = gain(true_targets, false_targets, root_mse)
            
            #  выбираем порог, на котором получается максимальный прирост качества
            if current_gain > best_gain:
                best_gain, best_t, best_index = current_gain, t, index

    return best_gain, best_t, best_index

In [22]:
import time
# Построение дерева с помощью рекурсивной функции

def build_tree(data, targets):

    gain, t, index = find_best_split(data, targets)

    #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
    if gain == 0:
        return Leaf(data, targets)

    true_data, false_data, true_targets, false_targets = split(data, targets, index, t)

    # Рекурсивно строим два поддерева
    true_branch = build_tree(true_data, true_targets)

#     print(time.time(), true_branch)
    false_branch = build_tree(false_data, false_targets)
    
#     print(time.time(), false_branch)
    
    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch)

### Функции предсказания

In [32]:
def predict_object(obj, node):

    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return predict_object(obj, node.true_branch)
    else:
        return predict_object(obj, node.false_branch)

In [33]:
def predict(data, tree):
    
    classes = []
    for obj in data:
        prediction = predict_object(obj, tree)
        classes.append(prediction)
    return classes

### Генерация данных и построение дерева

In [34]:
# сгенерируем данные
X, y = make_regression(n_samples=1000, n_features=2,n_informative=2, n_targets=1,
                noise=5.0, shuffle=True, random_state=11)

In [35]:
# Разобьем выборку на обучающую и тестовую

from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3, random_state=11)

In [36]:
# Построим дерево по обучающей выборке
my_tree = build_tree(train_X, train_y)

### Предсказания

In [38]:
train_answers = predict(train_X,my_tree)
train_r2_score = r2_score(train_y, train_answers)

In [40]:
test_answers = predict(test_X,my_tree)
test_r2_score = r2_score(test_y, test_answers)

In [42]:
train_r2_score, test_r2_score

(0.9945929939915166, 0.9741837038939142)